In [190]:
import pandas as pd
from openai import OpenAI

In [191]:
client = OpenAI(api_key="API_KEY")

In [192]:
df = pd.read_csv('../final/q2_with_summaries.csv')

In [193]:
df.columns

Index(['claim_id', 'claim', 'claim_date', 'speaker', 'reporting_source',
       'generated_question_1', 'judgement_1', 'summary_1', 'label_1', 'url_1',
       'context', 'generated_question_2', 'document_weight_question_2',
       'document_rank_question_2', 'document_url_question_2',
       'document_question_2', 'summary_2'],
      dtype='object')

### finding maximum siffix that we are going to use later

In [194]:
import re

# Assume merged_df is your DataFrame
df_columns = df.columns

def get_max_suffix(columns):
    max_suffix = 0
    for column in columns:
        match = re.search(r'(\d+)$', column)
        if match:
            suffix = int(match.group(1))
            if suffix > max_suffix:
                max_suffix = suffix
    return max_suffix

In [195]:
MAX_SUFFIX = get_max_suffix(df_columns)

In [196]:
print("The maximum suffix number is:", MAX_SUFFIX)

The maximum suffix number is: 2


In [197]:
df[f'label_{MAX_SUFFIX}'] = ""

In [198]:
## some changes for this df

In [199]:
# df.rename(columns={'summary': 'summary_2'}, inplace=True)

In [ ]:
df.rename(columns= {'url_2': 'document_url_question_2'}, inplace=True)


In [201]:
df.head()

,claim_id,claim,claim_date,speaker,reporting_source,generated_question_1,judgement_1,summary_1,label_1,document_url_question_1,context,generated_question_2,document_weight_question_2,document_rank_question_2,document_url_question_2,document_question_2,summary_2,label_2
0,2,Due to Imran Khan's criticism of Macron's comm...,31-10-2020,Consulate General Of Pakistan France,Twitter,Did French authorities cancel the visas of 183...,"No, the French authorities did not cancel the ...",The text contradicts the question. There is no...,Refuted,https://tribune.com.pk/story/1119830/diplomati...,{'generated question 1': 'Did French authoriti...,Did French authorities deport 118 Pakistani ci...,weighted,1,http://library.law.fsu.edu/Digital-Collections...,"~ p p e l l a n t , a native-born French c i ...",The provided text does not contain any informa...,
1,2,Due to Imran Khan's criticism of Macron's comm...,31-10-2020,Consulate General Of Pakistan France,Twitter,Did French authorities cancel the visas of 183...,"No, the French authorities did not cancel the ...",The text contradicts the question. There is no...,Refuted,https://tribune.com.pk/story/1119830/diplomati...,{'generated question 1': 'Did French authoriti...,Did French authorities deport 118 Pakistani ci...,weighted,2,https://en.wikipedia.org/wiki/Pakistanis_in_Fr...,"in Overseas DOM-TOM Réunion, French Guiana, Gu...",The provided text does not mention any informa...,
2,2,Due to Imran Khan's criticism of Macron's comm...,31-10-2020,Consulate General Of Pakistan France,Twitter,Did French authorities cancel the visas of 183...,"No, the French authorities did not cancel the ...",The text contradicts the question. There is no...,Refuted,https://tribune.com.pk/story/1119830/diplomati...,{'generated question 1': 'Did French authoriti...,Did French authorities deport 118 Pakistani ci...,weighted,3,https://www.france24.com/en/20130404-pakistan-...,Pakistan has deported three French nationals w...,The text does not provide any information rega...,
3,2,Due to Imran Khan's criticism of Macron's comm...,31-10-2020,Consulate General Of Pakistan France,Twitter,Did French authorities cancel the visas of 183...,"No, the French authorities did not cancel the ...",The text contradicts the question. There is no...,Refuted,https://tribune.com.pk/story/1119830/diplomati...,{'generated question 1': 'Did French authoriti...,Did French authorities deport 118 Pakistani ci...,weighted,4,https://www.indiatoday.in/world/story/pakistan...,Pakistan assembly votes to call back envoy fro...,The text does not mention anything about Frenc...,
4,2,Due to Imran Khan's criticism of Macron's comm...,31-10-2020,Consulate General Of Pakistan France,Twitter,Did French authorities cancel the visas of 183...,"No, the French authorities did not cancel the ...",The text contradicts the question. There is no...,Refuted,https://tribune.com.pk/story/1119830/diplomati...,{'generated question 1': 'Did French authoriti...,Did French authorities deport 118 Pakistani ci...,weighted,5,https://www.dawn.com/news/1447788,Undocumented Pakistani immigrants hide in the ...,The text contradicts the question as it does n...,


In [202]:
len(df)

10

In [203]:
df = df[df['claim_id'] != 'claim_id']

In [204]:
df['label_1'].value_counts()

label_1
Refuted    10
Name: count, dtype: int64

### Our prompts for further evaluation 

In [205]:
CHECK_PROMPT = """ Given the provided claim, question, and summary of the text, determine the most appropriate label for the relationship between the claim and the summary, assuming the summary serves as an answer to the question. The possible labels are:

1. **Refuted**: The summary clearly contradicts the claim.
2. **Supported**: The summary clearly supports the claim.
3. **Not Enough Evidence**: The summary does not provide sufficient evidence to either support or refute the claim. If there are no information in the text it means its Not Enough Evidence.

Claim is : [[CLAIM]]
Question is : [[QUESTION]]
Summary is: [[SUMMARY]]
Based on this information, what label can you give?
Just provide final label.
"""

In [206]:
CHOOSING_BEST_PROMPT = """
Based on the provided claim, question, and summaries, determine which summary provides the most relevant information related to the claim and question.

Claim: [[CLAIM]]
Question: [[QUESTION]]

Summaries:
[[SUMMARIES]]

Please choose the summary that best addresses the claim and question by providing its index.
"""

In [207]:
JUDGEMENT_PROMPT = """
Based on the provided claim, questions and summaries, answer the questions in a single answer.
Claim: [[CLAIM]]
Question 1: [[QUESTION_1]]
Summary 1: [[SUMMARY_1]]
Question 2: [[QUESTION_2]]
Summary 2: [[SUMMARY_2]]
Answer:
"""

In [208]:
CODE_DEMO_STOP = '''Claim = Superdrag and Collective Soul are both rock bands.
To validate the above claim, we have asked the following questions: 
Question 1 = Is Superdrag a rock band?
Answer 1 = Yes
Can we know whether the claim is true or false now?
Prediction = No, we cannot know. 

Claim = Superdrag and Collective Soul are both rock bands.
To validate the above claim, we have asked the following questions: 
Question 1 = Is Superdrag a rock band?
Answer 1 = Yes
Question 2 = Is Collective Soul a rock band?
Answer 2 = Yes
Can we know whether the claim is true or false now?
Prediction = Yes, we can know.

Claim = Jimmy Garcia lost by unanimous decision to a professional boxer that challenged for the WBO lightweight title in 1995. 
To validate the above claim, we have asked the following questions:
Question 1 = Who is the professional boxer that challenged for the WBO lightweight title in 1995? 
Answer 1 = Orzubek Nazarov
Can we know whether the claim is true or false now?
Prediction = No, we cannot know.

Claim = Jimmy Garcia lost by unanimous decision to a professional boxer that challenged for the WBO lightweight title in 1995. 
To validate the above claim, we have asked the following questions:
Question 1 = Who is the professional boxer that challenged for the WBO lightweight title in 1995? 
Answer 1 = Orzubek Nazarov
Question 2 = Did Jimmy Garcia lose by unanimous decision to Orzubek Nazarov?
Can we know whether the claim is true or false now?
Prediction = Yes, we can know.

Claim = The Swan of Catania was taught by the Italian composer Giovanni Furno.
To validate the above claim, we have asked the following questions: 
Question 1 = What is the nationality of Giovanni Furno?
Answer 1 = Italian
Can we know whether the claim is true or false now?
Prediction = No, we cannot know.

Claim = Lars Onsager won the Nobel prize when he was 30 years old.
To validate the above claim, we have asked the following questions:  
Question 1 = When Lars Onsager won the Nobel prize?
Answer 1 = 1968
Can we know whether the claim is true or false now?
Prediction = No, we cannot know.

Claim = Smith worked on the series The Handmaid's Tale that is based on a novel by Margaret Atwood. 
To validate the above claim, we have asked the following questions:
Question 1 = Which novel The Handmaid's Tale is based on?
Answer 1 = Margaret Atwood
Can we know whether the claim is true or false now?
Prediction = No, we cannot know.

Claim = Smith worked on the series The Handmaid's Tale that is based on a novel by Margaret Atwood. 
To validate the above claim, we have asked the following questions:
Question 1 = Which novel The Handmaid's Tale is based on?
Answer 1 = Margaret Atwood
Question 2 = Did Smith work on the series The Handmaid's Tale?
Answer 2 = Yes
Can we know whether the claim is true or false now?
Prediction = Yes, we can know.

Claim = The first season of the series The Handmaid's Tale was released in 2017.
To validate the above claim, we have asked the following questions:
Question 1 = When was the first season of the series The Handmaid's Tale released?
Answer 1 = 2017
Can we know whether the claim is true or false now?
Prediction = Yes, we can know.

Claim = [[CLAIM]]
To validate the above claim, we have asked the following questions:
[[QA_CONTEXTS]]
Can we know whether the claim is true or false now?
Prediction = '''

In [209]:
CODE_DEMO_SUBSEQUENT = '''Task: to verify a claim, we need to ask a series of simple questions. Here the task is given a claim and previous questions generate the following question to ask. 
This question should be:

- Simple with a single subject-verb-object structure.
- Specific and directly related to the key aspect of the claim that needs validation.

Claim = Superdrag and Collective Soul are both rock bands.
To validate the above claim, we need to ask the following simple questions sequentially: 
Question 1 = Is Superdrag a rock band?
Answer 1 = Yes
Question 2 = Is Collective Soul a rock band?

Claim = Jimmy Garcia lost by unanimous decision to a professional boxer that challenged for the WBO lightweight title in 1995. 
To validate the above claim, we need to ask the following simple questions sequentially: 
Question 1 = Who is the professional boxer that challenged for the WBO lightweight title in 1995? 
Answer 1 = Orzubek Nazarov
Question 2 = Did Jimmy Garcia lose by unanimous decision to Orzubek Nazarov?

Claim = The Swan of Catania was taught by the Italian composer Giovanni Furno.
To validate the above claim, we need to ask the following simple questions sequentially: 
Question 1 = What is the nationality of Giovanni Furno?
Answer 1 = Italian
Question 2 = Who was taught by Giovanni Furno?

Claim = Smith worked on the series The Handmaid's Tale that is based on a novel by Margaret Atwood.
To validate the above claim, we need to ask the following simple questions sequentially:
Question 1 = Which novel The Handmaid's Tale is based on?
Answer 1 = Margaret Atwood
Question 2 = Who worked on the series The Handmaid's Tale?

Claim = The Potomac River runs along the neighborhood where Ashley Estates Kavanaugh's wedding was held.
To validate the above claim, we need to ask the following simple questions sequentially:
Question 1 = Where was Ashley Estates Kavanaugh's wedding held?
Answer 1 = Christ Church in Georgetown
Question 2 = Which river runs along the Christ Church in Georgetown?

Claim = Ulrich Walter's employer is headquartered in Cologne.
To validate the above claim, we need to ask the following simple questions sequentially:
Question 1 = Who is Ulrich Walter's employer?
Answer 1 = University of Cologne
Question 2 = Where is the University of Cologne headquartered?

Claim = Lars Onsager won the Nobel prize when he was 30 years old.
To validate the above claim, we need to ask the following simple questions sequentially: 
Question 1 = When Lars Onsager won the Nobel prize?
Answer 1 = 1968
Question 2 = When was Lars Onsager born?

Claim = [[CLAIM]]
To validate the above claim, we need to ask the following simple questions sequentially: 
[[QA_CONTEXTS]]'''

In [210]:
MODEL="gpt-4o"

## Label generation 

In [211]:
# generating labels for each summary
# keeping it all in one row after each summaries 


def check_label(claim, question, summary):
    PR_template = CHECK_PROMPT
    example_input = PR_template.replace('[[CLAIM]]', claim.strip())
    example_input = example_input.replace('[[QUESTION]]', question.strip())
    example_input = example_input.replace('[[SUMMARY]]', summary.strip())
    completion = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are an expert annotator who assists in determining the relationship between a claim and a summary in the context of a given question. Your task is to label the summary as either Refuted, Supported, or Not Enough Evidence based on how it answers the question in relation to the claim. Just provide final label."},
            {"role": "user", "content": example_input}
        ]
    )
    return completion.choices[0].message.content


for index, row in df.iterrows():
    claim = row['claim']
    question = row[f'generated_question_{MAX_SUFFIX}']
    summary = row[f'summary_{MAX_SUFFIX}']
    label_column = f'label_{MAX_SUFFIX}'
    label = check_label(claim, question, summary)
    df.at[index, label_column] = label




In [212]:
df[f'label_{MAX_SUFFIX}'].value_counts()

label_2
Not Enough Evidence    8
Refuted                2
Name: count, dtype: int64

In [213]:
df.head(1)

,claim_id,claim,claim_date,speaker,reporting_source,generated_question_1,judgement_1,summary_1,label_1,document_url_question_1,context,generated_question_2,document_weight_question_2,document_rank_question_2,document_url_question_2,document_question_2,summary_2,label_2
0,2,Due to Imran Khan's criticism of Macron's comm...,31-10-2020,Consulate General Of Pakistan France,Twitter,Did French authorities cancel the visas of 183...,"No, the French authorities did not cancel the ...",The text contradicts the question. There is no...,Refuted,https://tribune.com.pk/story/1119830/diplomati...,{'generated question 1': 'Did French authoriti...,Did French authorities deport 118 Pakistani ci...,weighted,1,http://library.law.fsu.edu/Digital-Collections...,"~ p p e l l a n t , a native-born French c i ...",The provided text does not contain any informa...,Not Enough Evidence


In [237]:
unique_pairs = df[['claim_id', 'claim', 'claim_date', 'speaker', 'reporting_source', 'generated_question_1', 'document_url_question_1', 'summary_1', 'label_1', 'generated_question_2']].drop_duplicates()

unique_pairs['supported_summaries'] = None
unique_pairs['refuted_summaries'] = None
unique_pairs['not_enough_evidence_summaries'] = None
unique_pairs['supported_urls'] = None
unique_pairs['refuted_urls'] = None
unique_pairs['not_enough_evidence_urls'] = None

for idx, unique_pair in unique_pairs.iterrows():
    filtered_rows = df[
        (df['claim_id'] == unique_pair['claim_id']) &
        (df['claim'] == unique_pair['claim']) &
        (df['generated_question_1'] == unique_pair['generated_question_1']) &
        (df['document_url_question_1'] == unique_pair['document_url_question_1']) &
        (df['summary_1'] == unique_pair['summary_1']) &
        (df['label_1'] == unique_pair['label_1']) &
        (df['generated_question_2'] == unique_pair['generated_question_2'])
    ]
    # print(f'Rows for unique pair {idx}:')
    label_supported = filtered_rows[filtered_rows["label_2"] == "Supported"]["summary_2"]
    label_refuted = filtered_rows[filtered_rows["label_2"] == "Refuted"]["summary_2"]
    label_nee = filtered_rows[filtered_rows["label_2"] == "Not Enough Evidence"]["summary_2"]

    supported_url = filtered_rows[filtered_rows["label_2"] == "Supported"]["document_url_question_2"]
    refuted_url = filtered_rows[filtered_rows["label_2"] == "Refuted"]["document_url_question_2"]
    nee_url = filtered_rows[filtered_rows["label_2"] == "Not Enough Evidence"]["document_url_question_2"]

    unique_pairs = unique_pairs.copy()  # Make a copy to avoid SettingWithCopyWarning
    unique_pairs.at[idx, 'supported_summaries'] = list(label_supported)
    unique_pairs.at[idx, 'refuted_summaries'] = list(label_refuted)
    unique_pairs.at[idx, 'not_enough_evidence_summaries'] = list(label_nee)
    unique_pairs.at[idx, 'supported_urls'] = list(label_supported)
    unique_pairs.at[idx, 'refuted_urls'] = list(label_refuted)
    unique_pairs.at[idx, 'not_enough_evidence_urls'] = list(label_nee)
print(unique_pairs.columns)
processed_df = unique_pairs

Index(['claim_id', 'claim', 'claim_date', 'speaker', 'reporting_source',
       'generated_question_1', 'document_url_question_1', 'summary_1',
       'label_1', 'generated_question_2', 'supported_summaries',
       'refuted_summaries', 'not_enough_evidence_summaries', 'supported_urls',
       'refuted_urls', 'not_enough_evidence_urls'],
      dtype='object')


In [238]:
unique_pairs.head()

,claim_id,claim,claim_date,speaker,reporting_source,generated_question_1,document_url_question_1,summary_1,label_1,generated_question_2,supported_summaries,refuted_summaries,not_enough_evidence_summaries,supported_urls,refuted_urls,not_enough_evidence_urls
0,2,Due to Imran Khan's criticism of Macron's comm...,31-10-2020,Consulate General Of Pakistan France,Twitter,Did French authorities cancel the visas of 183...,https://tribune.com.pk/story/1119830/diplomati...,The text contradicts the question. There is no...,Refuted,Did French authorities deport 118 Pakistani ci...,[],[The summarized information contradicts the qu...,[The provided text does not contain any inform...,[],[The summarized information contradicts the qu...,[The provided text does not contain any inform...


In [239]:
#finding best summary and saving question and answer pair
import pandas as pd
import re



# Imitate GPT call to always return the first index
def choose_best_summary(claim, question, summaries):
    if not summaries:
        return None, None

    summaries_text = "\n".join([f"[[{i}]] - {summary}" for i, summary in enumerate(summaries)])
    prompt = CHOOSING_BEST_PROMPT.replace("[[CLAIM]]", claim).replace("[[QUESTION]]", question).replace("[[SUMMARIES]]", summaries_text)
    
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an expert annotator who assists in determining best and most informative summary based on the provided claim, question and text."},
            {"role": "user", "content": prompt}
        ]
    )
    
    content = completion.choices[0].message.content.strip()
    
    # Extract the index using regex
    match = re.search(r'\[\[(\d+)\]\]', content)
    if not match:
        match = re.search(r'(\d+)', content)
    
    if match:
        best_summary_index = int(match.group(1))
        if best_summary_index < len(summaries):
            return summaries[best_summary_index], best_summary_index
        else:
#             raise ValueError(f"Extracted index {best_summary_index} is out of range for summaries: {content}")
            return summaries[0],0
    else:
#         raise ValueError(f"Invalid index format in GPT response: {content}")
        return summaries[0],0


# Function to apply the filtering logic and choose the best summary
def process_row(row):
    supported_summaries = row['supported_summaries'] if isinstance(row['supported_summaries'], list) else eval(row['supported_summaries'])
    supported_urls = row['supported_urls'] if isinstance(row['supported_urls'], list) else eval(row['supported_urls'])
    refuted_summaries = row['refuted_summaries'] if isinstance(row['refuted_summaries'], list) else eval(row['refuted_summaries'])
    refuted_urls = row['refuted_urls'] if isinstance(row['refuted_urls'], list) else eval(row['refuted_urls'])
    not_enough_evidence_summaries = row['not_enough_evidence_summaries'] if isinstance(row['not_enough_evidence_summaries'], list) else eval(row['not_enough_evidence_summaries'])
    not_enough_evidence_urls = row['not_enough_evidence_urls'] if isinstance(row['not_enough_evidence_urls'], list) else eval(row['not_enough_evidence_urls'])
    
    claim_id = row['claim_id']
    claim = row['claim']
    question = row['generated_question_2']
    claim_date = row['claim_date']
    speaker = row['speaker']
    reporting_source = row['reporting_source']
    
    results = []

    if supported_summaries:
        if len(supported_summaries) == 1:
            best_supported_summary = supported_summaries[0]
            best_supported_url = supported_urls[0]
        else:
            best_supported_summary, best_supported_index = choose_best_summary(claim, question, supported_summaries)
            best_supported_url = supported_urls[best_supported_index]
        results.append({
            'claim_id': claim_id,
            'claim': claim,
            'claim_date': claim_date,
            'speaker': speaker,
            'reporting_source': reporting_source,
            'generated_question_1': row["generated_question_1"], 
            'document_url_question_1': row["document_url_question_1"], 
            'summary_1': row["summary_1"], 
            'label_1': row["label_1"],
            'generated_question_2': question,
            'document_url_question_2': best_supported_url, 
            'summary_2': best_supported_summary,
            'label_2': 'Supported'
        })
        
    if refuted_summaries:
        if len(refuted_summaries) == 1:
            best_refuted_summary = refuted_summaries[0]
            best_refuted_url = refuted_urls[0]
        else:
            best_refuted_summary, best_refuted_index = choose_best_summary(claim, question, refuted_summaries)
            best_refuted_url = refuted_urls[best_refuted_index]
        results.append({
            'claim_id': claim_id,
            'claim': claim,
            'claim_date': claim_date,
            'speaker': speaker,
            'reporting_source': reporting_source,
            'generated_question_1': row["generated_question_1"], 
            'document_url_question_1': row["document_url_question_1"], 
            'summary_1': row["summary_1"], 
            'label_1': row["label_1"],
            'generated_question_2': question,
            'document_url_question_2': best_refuted_url, 
            'summary_2': best_refuted_summary,
            'label_2': 'Refuted'
        })
        
    if not results and not_enough_evidence_summaries:
        if len(not_enough_evidence_summaries) == 1:
            best_not_enough_evidence_summary = not_enough_evidence_summaries[0]
            best_not_enough_evidence_url = not_enough_evidence_urls[0]
        else:
            best_not_enough_evidence_summary, best_not_enough_evidence_index = choose_best_summary(claim, question, not_enough_evidence_summaries)
            best_not_enough_evidence_url = not_enough_evidence_urls[best_not_enough_evidence_index]
        results.append({

            'claim_id': claim_id,
            'claim': claim,
            'claim_date': claim_date,
            'speaker': speaker,
            'reporting_source': reporting_source,
            'generated_question_1': row["generated_question_1"], 
            'document_url_question_1': row["document_url_question_1"], 
            'summary_1': row["summary_1"], 
            'label_1': row["label_1"],
            'generated_question_2': question,
            'document_url_question_2': best_not_enough_evidence_url, 
            'summary_2': best_not_enough_evidence_summary,
            'label_2': 'Not Enough Evidence'
        })

    return pd.DataFrame(results)

# Create an empty DataFrame to store the results
final_df = pd.DataFrame(columns=['claim_id', 'claim', 'generated_question_1', 'document_url_question_1', 'summary_1', 'label_1', 'generated_question_2', 'summary_2', 'label_2'])

# Apply the processing function to each row
for index, row in processed_df.iterrows():
    processed_row_df = process_row(row)
    final_df = pd.concat([final_df, processed_row_df], ignore_index=True)



In [240]:
final_df.head(2)

,claim_id,claim,generated_question_1,document_url_question_1,summary_1,label_1,generated_question_2,summary_2,label_2,claim_date,speaker,reporting_source,document_url_question_2
0,2,Due to Imran Khan's criticism of Macron's comm...,Did French authorities cancel the visas of 183...,https://tribune.com.pk/story/1119830/diplomati...,The text contradicts the question. There is no...,Refuted,Did French authorities deport 118 Pakistani ci...,The summarized information contradicts the que...,Refuted,31-10-2020,Consulate General Of Pakistan France,Twitter,The summarized information contradicts the que...


In [241]:


#getting judgement to the summary and question based on the summary

def get_judgement(claim, question_1, summary_1, question_2, summary_2):
    judgement_input = JUDGEMENT_PROMPT.replace('[[CLAIM]]', claim.strip())
    judgement_input = judgement_input.replace('[[QUESTION_1]]', question_1.strip())
    judgement_input = judgement_input.replace('[[SUMMARY_2]]', summary_1.strip())
    judgement_input = judgement_input.replace('[[QUESTION_1]]', question_2.strip())
    judgement_input = judgement_input.replace('[[SUMMARY_2]]', summary_2.strip())
    
    completion = client.chat.completions.create(
      model=MODEL,
      messages=[
        {"role": "system", "content": "You are an expert annotator who assist in providing answer based on the provided information of claim, two questions and corresponding extracted summaries of the information."},
        {"role": "user", "content":judgement_input}
      ]
    )
    
    answer = completion.choices[0].message.content
    return answer

# Add a new column for judgements
final_df['judgement'] = final_df.apply(lambda row: get_judgement(row['claim'], row['generated_question_1'], row['summary_1'], row['generated_question_2'], row['summary_2']), axis=1)



In [242]:
final_df[["claim", "generated_question_1", "summary_1", "generated_question_2", "summary_2", "judgement"]].head(1)

,claim,generated_question_1,summary_1,generated_question_2,summary_2,judgement
0,Due to Imran Khan's criticism of Macron's comm...,Did French authorities cancel the visas of 183...,The text contradicts the question. There is no...,Did French authorities deport 118 Pakistani ci...,The summarized information contradicts the que...,"No, there is no mention of French authorities ..."


In [243]:
def get_verification_status(claim, question_1, answer_1, question_2, answer_2, judgement):
    qa_contexts_txt = f'Question 1 = {question_1}\nAnswer 1 = {answer_1}\n'
    qa_contexts_txt = qa_contexts_txt + f'Question 2 = {question_2}\nAnswer 2 = {answer_2}\n'
    qa_contexts_txt = qa_contexts_txt + f'Summarised Judgement = {judgement}\n'
    example = CODE_DEMO_STOP.replace('[[CLAIM]]', claim.strip())
    example = example.replace('[[QA_CONTEXTS]]', qa_contexts_txt.strip())
    
    completion = client.chat.completions.create(
      model=MODEL,
      messages=[
        {"role": "system", "content": "You are an expert annotator who assist in telling are we having enough information to verify claim or not based on the provided claim, questions and answers. Answer only yes or no"},
        {"role": "user", "content":example}
      ]
    )
    can_we_continue = completion.choices[0].message.content
    return can_we_continue

# Add a new column for verification status
final_df['verification_status'] = final_df.apply(lambda row: get_verification_status(row['claim'], row['generated_question_1'], row['summary_1'], row['generated_question_2'], row['summary_2'], row['judgement']), axis=1)


In [244]:
final_df.head()

,claim_id,claim,generated_question_1,document_url_question_1,summary_1,label_1,generated_question_2,summary_2,label_2,claim_date,speaker,reporting_source,document_url_question_2,judgement,verification_status
0,2,Due to Imran Khan's criticism of Macron's comm...,Did French authorities cancel the visas of 183...,https://tribune.com.pk/story/1119830/diplomati...,The text contradicts the question. There is no...,Refuted,Did French authorities deport 118 Pakistani ci...,The summarized information contradicts the que...,Refuted,31-10-2020,Consulate General Of Pakistan France,Twitter,The summarized information contradicts the que...,"No, there is no mention of French authorities ...","Yes, we can know."


In [245]:
# and now generating new question
def generate_followup_question(claim, row):
    qa_contexts_txt = f"Question 1 = {row['generated_question_1']}\nAnswer 1 = {row['summary_1']}\n"
    qa_contexts_txt = qa_contexts_txt + f"Question 2 = {row['generated_question_2']}\nAnswer 2 = {row['summary_2']}\n"
    qa_contexts_txt = qa_contexts_txt + f"Summarised Judgement = {row['judgement']}\n"
    example_followup = CODE_DEMO_SUBSEQUENT.replace('[[CLAIM]]', claim.strip())
    example_followup = example_followup.replace('[[QA_CONTEXTS]]', qa_contexts_txt.strip())
    
    completion = client.chat.completions.create(
      model=MODEL,
      messages=[
        {"role": "system", "content": "You are an expert annotator who assist in generating new question that would be required to validate claim, based on the claim, previous questions and answers. Return only question."},
        {"role": "user", "content":example_followup}
      ] 
    )
    followup_question = completion.choices[0].message.content
    return followup_question

# Create a new DataFrame to store the results

new_rows = []

for index, row in final_df.iterrows():
    if not re.search(r'\byes\b', row['verification_status'], re.IGNORECASE):
        claim = row['claim']
        new_question = generate_followup_question(claim, row)
        
        new_row = row
        new_row['generated_question_3'] = new_question
        
        new_rows.append(new_row)

# Convert the list of new rows to a DataFrame
new_df = pd.DataFrame(new_rows)


In [246]:
new_df

,claim_id,claim,generated_question_1,document_url_question_1,summary_1,label_1,generated_question_2,summary_2,label_2,claim_date,speaker,reporting_source,document_url_question_2,judgement,verification_status,generated_question_3
0,2,Due to Imran Khan's criticism of Macron's comm...,Did French authorities cancel the visas of 183...,https://tribune.com.pk/story/1119830/diplomati...,The text contradicts the question. There is no...,Refuted,Did French authorities deport 118 Pakistani ci...,The summarized information contradicts the que...,Refuted,31-10-2020,Consulate General Of Pakistan France,Twitter,The summarized information contradicts the que...,"No, there is no mention of French authorities ...","Yes, we can know.",Did Imran Khan criticize Macron's comments on ...


In [247]:
new_df.to_csv('../final/q3_samples_dev.csv', index = False)

In [1]:
remaining_rows = []

for index, row in final_df.iterrows():
    if re.search(r'\byes\b', row['verification_status'], re.IGNORECASE):
        remaining_row = {
            'claim_id': row['claim_id'],
            'claim': row['claim'],
            'claim_date': row['claim_date'],
            'speaker': row['speaker'],
            'reporting_source': row['reporting_source'],
            'generated_question_1': row['generated_question_1'],
            'summary_1': row['summary_1'],
            'label_1': row['label_1'],
            'document_url_question_1': row['document_url_question_1'],
            'generated_question_2': row['generated_question_2'],
            'summary_2': row['summary_2'],
            'label_2': row['label_2'],
            'document_url_question_2': row['document_url_question_2'],
            'judgement': row['judgement'].replace("\n", " ").strip(),
        }
        
        remaining_rows.append(remaining_row)
remaining_df = pd.DataFrame(remaining_rows)

NameError: name 'final_df' is not defined

In [2]:
remaining_df.to_csv('../final/out_csvs/q2_enough_generated.csv', index  = False)

NameError: name 'remaining_df' is not defined

In [477]:
# processed_df.drop('label_2', axis = 1, inplace = True)

In [478]:
# for index, row in processed_df.iterrows():
#     row['label_1'] = row['label_1'][0]

In [479]:
# processed_df.columns